In [45]:
import re
import numpy as np
import pandas as pd
import pickle

In [46]:
pd.options.mode.chained_assignment = None  
pd.options.display.max_columns=None
pd.options.display.max_rows=100
pd.set_option('display.max_colwidth', None)

In [47]:
df = pd.read_csv('../datos/datos_sin_unos.csv', index_col = 0)
df.head(2)

,Unnamed: 0,age,gender,country,continent,job_title,work_activities,size_DA_dept,years_programming,first_language_rec,dev_language,primary_data_tool,IDE,big_data,BI_tools,visualisation,ML,notebooks_KaggleNotebooks,notebooks_ColabNotebooks,notebooks_AzureNotebooks,notebooks_Paperspace/Gradient,notebooks_Binder/JupyterHub,notebooks_CodeOcean,notebooks_IBMWatsonStudio,notebooks_AmazonSagemakerStudioNotebooks,notebooks_AmazonEMRNotebooks,notebooks_GoogleCloudNotebooks(AIPlatform/VertexAI),notebooks_GoogleCloudDatalab,notebooks_DatabricksCollaborativeNotebooks,notebooks_Zeppelin/ZeplNotebooks,notebooks_DeepnoteNotebooks,notebooks_ObservableNotebooks,notebooks_None,notebooks_Other,sharing_PlotlyDash,sharing_Streamlit,sharing_NBViewer,sharing_GitHub,sharing_Personalblog,sharing_Kaggle,sharing_Colab,sharing_Shiny,sharing_does_not_share,sharing_Other,dev_language_Javascript,dev_language_None,dev_language_R,dev_language_Bash,dev_language_Python,dev_language_Other,dev_language_C,dev_language_MATLAB,dev_language_Julia,dev_language_Java,dev_language_C++,dev_language_Swift,dev_language_SQL,IDE_None,IDE_Sublime Text,IDE_Vim / Emacs,IDE_RStudio,IDE_Jupyter (JupyterLab; Jupyter Notebooks; etc),IDE_Other,IDE_Visual Studio,IDE_Notepad++,IDE_Visual Studio Code (VSCode),IDE_Spyder,IDE_PyCharm,IDE_MATLAB,IDE_Jupyter Notebook,visualisation_None,visualisation_Matplotlib,visualisation_Other,visualisation_Seaborn,visualisation_Altair,visualisation_Shiny,visualisation_D3 js,visualisation_Bokeh,visualisation_Leaflet / Folium,visualisation_Plotly / Plotly Express,visualisation_Geoplotlib,visualisation_Ggplot / ggplot2,activities_ML_prototypes,activities_run_ML,activities_ML_research,activities_Other,activities_improve_ML,activities_None,activities_analyze_data,activities_data_infrastructure,big_data_Google Cloud Firestore,big_data_Google Cloud BigTable,big_data_Google Cloud Spanner,big_data_PostgreSQL,big_data_Microsoft Azure Cosmos DB,big_data_SQLite,big_data_Amazon DynamoDB,big_data_None,big_data_Microsoft Azure SQL Database,big_data_Amazon RDS,big_data_Other,big_data_Snowflake,big_data_Amazon Aurora,big_data_Google Cloud SQL,big_data_Microsoft SQL Server,big_data_Oracle Database,big_data_Google Cloud BigQuery,big_data_Amazon Redshift,big_data_IBM Db2,big_data_MongoDB,big_data_MySQL,BI_tools_None,BI_tools_Tableau,BI_tools_Looker,BI_tools_Google Data Studio,BI_tools_TIBCO Spotfire,BI_tools_Other,BI_tools_Qlik,BI_tools_Tableau CRM,BI_tools_SAP Analytics Cloud,BI_tools_Sisense,BI_tools_Microsoft Power BI,BI_tools_Amazon QuickSight,BI_tools_Salesforce,BI_tools_Microsoft Azure Synapse,BI_tools_Domo,BI_tools_Thoughtspot,BI_tools_Alteryx
0,0,50-54,Man,India,Asia,Other,None of these activities are an important part of my role at work,3-4,5-10 years,Python,"Python, R","Local development environments (RStudio, JupyterLab, etc.)",Vim / Emacs,PostgreSQL,None,"Matplotlib ,Seaborn ,Ggplot / ggplot2 ,Shiny ,Leaflet / Folium",5-10 years,NaN,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GitHub,NaN,Kaggle,NaN,NaN,NaN,NaN,NaN,NaN,R,NaN,Python,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vim / Emacs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matplotlib,NaN,Seaborn,NaN,Shiny,NaN,NaN,Leaflet / Folium,NaN,NaN,Ggplot / ggplot2,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,PostgreSQL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,50-54,Man,Indonesia,Asia,Program/Project Manager,Build and/or run the data infrastructure that my business uses for storing; analyzing; and operationalizing data,1-2,20+ years,Python,"SQL, C, C++, Java","Advanced statistical software (SPSS, SAS, etc.)","Notepad++,Jupyter Notebook",NaN,NaN,Matplotlib,< 1 year,Kaggle Notebooks,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,NaN,NaN,Java,C++,NaN,SQL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Notepad+

In [48]:
df_da = df[(df["job_title"] == "Business Analyst") | (df["job_title"] == "Data Analyst")]

In [49]:
with open('../datos/diccionario_respuestas.pkl', 'rb') as f:
    diccionario_respuestas = pickle.load(f)

In [50]:
lista_columnas_combinar = ['dev_language_', 'IDE_', 'visualisation_', 'activities_', 'big_data_', 'BI_tools_', 'sharing_', 'notebooks_']

In [51]:
df_da.replace(np.nan, "", inplace=True)

In [52]:
def replace_punto_coma(col):
    return re.sub(r";+", ";", col)

In [53]:
def replace_none(col):
    return re.sub(r"None;", ";", col)

In [70]:
def columna_combo(dataframe, lista_columnas, nombre):
    lista = []
    dataframe[nombre] = ""

    for col in dataframe.columns:
        for i in lista_columnas:
            if i in col:
                lista.append(col)
    
    set_col = set(lista)
    print(set_col)
    for col in set_col:
        dataframe[nombre] += df_da[col] + ";"

    dataframe[nombre] = dataframe[nombre].apply(replace_punto_coma)
    
    dataframe[nombre] = dataframe[nombre].apply(replace_none)

    dataframe[nombre] = dataframe[nombre].apply(lambda x: x.strip(";"))
    
    dataframe[nombre] = dataframe[nombre].apply(replace_punto_coma)

    df_combo_counts = dataframe[nombre].value_counts().reset_index()

    df_combo_counts.columns = ['tools', 'count']
    
    display(df_combo_counts.head(20))
    
    return df_combo_counts

In [55]:
# def columna_combo(lista_columnas, nombre):

#     df_da[nombre] = ""

#     for col in df_da.columns:
#         for i in lista_columnas:
#             if i in col:
#                 df_da[nombre] += df_da[col] + ";"

#     df_da[nombre] = df_da[nombre].apply(replace_punto_coma)
    
#     df_da[nombre] = df_da[nombre].apply(replace_none)

#     df_da[nombre] = df_da[nombre].apply(lambda x: x.strip(";"))
    
#     df_da[nombre] = df_da[nombre].apply(replace_punto_coma)

#     df_combo_counts = df_da[nombre].value_counts().reset_index()

#     df_combo_counts.columns = ['tools', 'count']
    
#     display(df_combo_counts.head(20))
    
#     return df_combo_counts

In [56]:
df_combo_all = columna_combo(df_da, lista_columnas_combinar, 'combo_all')

{'sharing_Other', 'dev_language_Swift', 'BI_tools_Thoughtspot', 'visualisation_Shiny', 'big_data_Google Cloud Firestore', 'big_data_Other', 'big_data_Google Cloud SQL', 'visualisation_Seaborn', 'BI_tools_Alteryx', 'IDE_Visual Studio Code (VSCode)', 'dev_language_Other', 'visualisation_D3 js', 'BI_tools_Microsoft Azure Synapse', 'big_data_None', 'visualisation_Ggplot / ggplot2', 'BI_tools_Other', 'notebooks_CodeOcean', 'big_data_IBM Db2', 'big_data_Google Cloud Spanner', 'notebooks_GoogleCloudNotebooks(AIPlatform/VertexAI)', 'BI_tools_Sisense', 'dev_language_Javascript', 'visualisation_Altair', 'notebooks_None', 'notebooks_AmazonSagemakerStudioNotebooks', 'IDE_Visual Studio', 'notebooks_DatabricksCollaborativeNotebooks', 'sharing_NBViewer', 'activities_None', 'visualisation_Plotly / Plotly Express', 'dev_language_C++', 'big_data_Google Cloud BigQuery', 'notebooks_AzureNotebooks', 'IDE_Notepad++', 'visualisation_Bokeh', 'BI_tools_TIBCO Spotfire', 'activities_ML_prototypes', 'big_data_Mic

/tmp/ipykernel_4263/2073644075.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_da[nombre] = ""


,tools,count
0,analyze_data,136
1,,37
2,Other,21
3,analyze_data;data_infrastructure,16
4,data_infrastructure,11
5,ML_prototypes;analyze_data;improve_ML;ML_research;data_infrastructure;run_ML,8
6,analyze_data;ML_research,7
7,run_ML,5
8,Seaborn;Jupyter Notebook;Python;Matplotlib; Kaggle Notebooks,4
9,analyze_data;run_ML,4


In [57]:
df_dev_language_vis = columna_combo(df_da, ['dev_language_', 'visualisation_'], 'combo_dev_lang_vis')

{'dev_language_Swift', 'visualisation_Plotly / Plotly Express', 'dev_language_C++', 'visualisation_Shiny', 'dev_language_R', 'visualisation_Seaborn', 'dev_language_Other', 'visualisation_D3 js', 'visualisation_Bokeh', 'dev_language_Java', 'visualisation_Leaflet / Folium', 'visualisation_Ggplot / ggplot2', 'dev_language_Python', 'dev_language_Julia', 'visualisation_None', 'visualisation_Matplotlib', 'dev_language_None', 'dev_language_C', 'dev_language_SQL', 'dev_language_MATLAB', 'visualisation_Geoplotlib', 'dev_language_Javascript', 'visualisation_Other', 'visualisation_Altair', 'dev_language_Bash'}


/tmp/ipykernel_4263/2073644075.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_da[nombre] = ""


,tools,count
0,,314
1,Seaborn;Python;Matplotlib;SQL,223
2,Seaborn;Python;Matplotlib,154
3,Plotly / Plotly Express;Seaborn;Python;Matplotlib;SQL,94
4,Python,72
5,Python;Matplotlib;SQL,72
6,Python;Matplotlib,72
7,SQL,69
8,Python;SQL,54
9,Plotly / Plotly Express;Seaborn;Python;Matplotlib,52


In [58]:
#['dev_language_', 'IDE_', 'visualisation_', 'activities_', 'big_data_', 'BI_tools_', 'sharing_', 'notebooks_']

In [59]:
df_dev_language_BI = columna_combo(df_da, ['dev_language_', 'BI_tools_'], 'combo_dev_lang_BI')

{'BI_tools_Microsoft Power BI', 'dev_language_Swift', 'dev_language_C++', 'BI_tools_Thoughtspot', 'dev_language_R', 'BI_tools_Salesforce', 'BI_tools_Alteryx', 'dev_language_Other', 'BI_tools_Looker', 'BI_tools_TIBCO Spotfire', 'BI_tools_Tableau', 'dev_language_Java', 'BI_tools_Microsoft Azure Synapse', 'BI_tools_Qlik', 'BI_tools_Amazon QuickSight', 'dev_language_Python', 'dev_language_Julia', 'BI_tools_Domo', 'BI_tools_SAP Analytics Cloud', 'dev_language_None', 'dev_language_C', 'BI_tools_Other', 'dev_language_SQL', 'dev_language_MATLAB', 'BI_tools_None', 'BI_tools_Sisense', 'BI_tools_Google Data Studio', 'dev_language_Javascript', 'BI_tools_Tableau CRM', 'dev_language_Bash'}


/tmp/ipykernel_4263/2073644075.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_da[nombre] = ""


,tools,count
0,Python,326
1,Python;SQL,314
2,,311
3,R;Python;SQL,152
4,R;Python,86
5,Tableau;Python;SQL,62
6,Microsoft Power BI;Python;SQL,59
7,SQL,55
8,Microsoft Power BI;Tableau;Python;SQL,45
9,Tableau;Python,40


In [60]:
df_dev_language_big_data = columna_combo(df_da, ['dev_language_', 'big_data_'], 'combo_dev_lang_big_data')

{'big_data_Microsoft SQL Server', 'big_data_SQLite', 'dev_language_Swift', 'dev_language_C++', 'big_data_Google Cloud Firestore', 'big_data_Other', 'big_data_MySQL', 'big_data_Oracle Database', 'big_data_Google Cloud BigQuery', 'big_data_Google Cloud SQL', 'dev_language_R', 'dev_language_Other', 'big_data_Amazon Redshift', 'dev_language_Java', 'big_data_Google Cloud BigTable', 'big_data_None', 'big_data_Microsoft Azure Cosmos DB', 'dev_language_Python', 'big_data_PostgreSQL', 'dev_language_Julia', 'dev_language_None', 'dev_language_C', 'big_data_Amazon RDS', 'dev_language_SQL', 'big_data_Microsoft Azure SQL Database', 'dev_language_MATLAB', 'big_data_MongoDB', 'big_data_IBM Db2', 'big_data_Google Cloud Spanner', 'big_data_Amazon DynamoDB', 'big_data_Amazon Aurora', 'dev_language_Javascript', 'dev_language_Bash', 'big_data_Snowflake'}


/tmp/ipykernel_4263/2073644075.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_da[nombre] = ""


,tools,count
0,Python,328
1,,321
2,Python;SQL,293
3,R;Python;SQL,144
4,R;Python,94
5,SQL,54
6,R,39
7,MySQL;Python;SQL,37
8,Other;Python;SQL,36
9,MySQL;Python,33


In [61]:
df_dev_language_notebooks = columna_combo(df_da, ['dev_language_', 'notebooks_'], 'combo_dev_lang_notebooks_')

{'notebooks_DatabricksCollaborativeNotebooks', 'dev_language_Swift', 'dev_language_C++', 'dev_language_R', 'combo_dev_lang_notebooks_', 'notebooks_ColabNotebooks', 'notebooks_DeepnoteNotebooks', 'notebooks_AmazonEMRNotebooks', 'notebooks_Paperspace/Gradient', 'dev_language_Other', 'notebooks_GoogleCloudDatalab', 'notebooks_AzureNotebooks', 'dev_language_Bash', 'notebooks_Zeppelin/ZeplNotebooks', 'dev_language_Java', 'notebooks_Other', 'dev_language_Python', 'notebooks_Binder/JupyterHub', 'dev_language_Julia', 'notebooks_IBMWatsonStudio', 'dev_language_None', 'dev_language_C', 'dev_language_SQL', 'notebooks_CodeOcean', 'notebooks_ObservableNotebooks', 'dev_language_MATLAB', 'notebooks_KaggleNotebooks', 'dev_language_Javascript', 'notebooks_GoogleCloudNotebooks(AIPlatform/VertexAI)', 'notebooks_AmazonSagemakerStudioNotebooks', 'notebooks_None'}


/tmp/ipykernel_4263/2073644075.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_da[nombre] = ""


,tools,count
0,,318
1,Python;SQL,179
2,Python,132
3,Python;SQL; Kaggle Notebooks,94
4,Colab Notebooks;Python;SQL,84
5,Python; Kaggle Notebooks,78
6,R;R;Python;SQL,74
7,Colab Notebooks;Python;SQL; Kaggle Notebooks,73
8,Colab Notebooks;Python,72
9,SQL,66


In [64]:
df_combo_all_tools = columna_combo(df_da, ['dev_language_', 'IDE_', 'visualisation_', 'big_data_', 'BI_tools_', 'notebooks_'], 'combo_all_tools')

{'dev_language_Swift', 'BI_tools_Thoughtspot', 'visualisation_Shiny', 'big_data_Google Cloud Firestore', 'big_data_Other', 'big_data_Google Cloud SQL', 'visualisation_Seaborn', 'BI_tools_Alteryx', 'IDE_Visual Studio Code (VSCode)', 'dev_language_Other', 'visualisation_D3 js', 'BI_tools_Microsoft Azure Synapse', 'big_data_None', 'visualisation_Ggplot / ggplot2', 'BI_tools_Other', 'notebooks_CodeOcean', 'big_data_IBM Db2', 'big_data_Google Cloud Spanner', 'notebooks_GoogleCloudNotebooks(AIPlatform/VertexAI)', 'BI_tools_Sisense', 'dev_language_Javascript', 'visualisation_Altair', 'notebooks_None', 'IDE_Visual Studio', 'notebooks_DatabricksCollaborativeNotebooks', 'visualisation_Plotly / Plotly Express', 'dev_language_C++', 'big_data_Google Cloud BigQuery', 'notebooks_AzureNotebooks', 'IDE_Notepad++', 'visualisation_Bokeh', 'BI_tools_TIBCO Spotfire', 'big_data_Microsoft Azure Cosmos DB', 'BI_tools_Amazon QuickSight', 'IDE_Jupyter Notebook', 'notebooks_Binder/JupyterHub', 'dev_language_Juli

,tools,count
0,,288
1,Ggplot / ggplot2;R;R;R;RStudio,9
2,Seaborn;Jupyter Notebook;Python;Python;Matplotlib,9
3,Seaborn;Jupyter Notebook;SQL;Python;SQL;Python;Matplotlib,8
4,Other;Other,7
5,Seaborn;Jupyter Notebook;Colab Notebooks;Python; Kaggle Notebooks;Colab Notebooks;Python;Matplotlib; Kaggle Notebooks,7
6,Python;Python;PyCharm,7
7,Seaborn;Jupyter Notebook;Python; Kaggle Notebooks;Python;Matplotlib; Kaggle Notebooks,6
8,SQL;SQL,6
9,Seaborn;Jupyter Notebook;SQL;Python;SQL; Kaggle Notebooks;Python;Matplotlib; Kaggle Notebooks,6


In [65]:
df_combo_R_vis = columna_combo(df_da, ['dev_language_R', 'visualisation_'], 'combo_R_vis')

{'visualisation_Seaborn', 'visualisation_Bokeh', 'visualisation_Leaflet / Folium', 'visualisation_Plotly / Plotly Express', 'visualisation_Geoplotlib', 'visualisation_Ggplot / ggplot2', 'visualisation_Shiny', 'visualisation_None', 'visualisation_Matplotlib', 'visualisation_Other', 'visualisation_D3 js', 'visualisation_Altair', 'dev_language_R'}


/tmp/ipykernel_4263/2073644075.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_da[nombre] = ""


,tools,count
0,,686
1,Seaborn;Matplotlib,541
2,Matplotlib,273
3,Seaborn;Plotly / Plotly Express;Matplotlib,230
4,Ggplot / ggplot2;R,111
5,Seaborn;Ggplot / ggplot2;Matplotlib;R,84
6,R,79
7,Seaborn;Matplotlib;R,61
8,Seaborn;Plotly / Plotly Express;Ggplot / ggplot2;Matplotlib;R,59
9,Seaborn;Ggplot / ggplot2;Matplotlib,53


In [66]:
df_combo_data_vis_bi = columna_combo(df_da, ['visualisation_', 'big_data_', 'BI_tools_',], 'data_vis_bi')

{'big_data_Microsoft SQL Server', 'BI_tools_Microsoft Power BI', 'big_data_SQLite', 'visualisation_Plotly / Plotly Express', 'BI_tools_Thoughtspot', 'visualisation_Shiny', 'big_data_Google Cloud Firestore', 'big_data_Other', 'big_data_MySQL', 'big_data_Oracle Database', 'big_data_Google Cloud BigQuery', 'big_data_Google Cloud SQL', 'BI_tools_Salesforce', 'visualisation_Seaborn', 'BI_tools_Alteryx', 'BI_tools_Looker', 'visualisation_D3 js', 'visualisation_Bokeh', 'big_data_Amazon Redshift', 'BI_tools_Tableau', 'BI_tools_TIBCO Spotfire', 'BI_tools_Microsoft Azure Synapse', 'visualisation_Leaflet / Folium', 'big_data_Google Cloud BigTable', 'big_data_None', 'BI_tools_Qlik', 'big_data_Microsoft Azure Cosmos DB', 'BI_tools_Amazon QuickSight', 'visualisation_Ggplot / ggplot2', 'big_data_PostgreSQL', 'visualisation_None', 'visualisation_Matplotlib', 'BI_tools_Domo', 'BI_tools_SAP Analytics Cloud', 'big_data_Amazon RDS', 'BI_tools_Other', 'big_data_Microsoft Azure SQL Database', 'big_data_Mong

/tmp/ipykernel_4263/2073644075.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_da[nombre] = ""


,tools,count
0,,579
1,Seaborn;Matplotlib,293
2,Matplotlib,155
3,Plotly / Plotly Express;Seaborn;Matplotlib,124
4,Seaborn;Ggplot / ggplot2;Matplotlib,74
5,Ggplot / ggplot2,61
6,Plotly / Plotly Express;Seaborn;Ggplot / ggplot2;Matplotlib,52
7,Other,34
8,Seaborn,29
9,Plotly / Plotly Express;Matplotlib,27


In [67]:
lista_drop = ['big_data_None',
 'dev_language_Javascript',
 'dev_language_Other',
 'dev_language_C++',
 'sharing_Colab',
 'notebooks_GoogleCloudDatalab',
 'IDE_Sublime Text',
 'big_data_SQLite',
 'notebooks_GoogleCloudNotebooks(AIPlatform/VertexAI)',
 'notebooks_Binder/JupyterHub',
 'visualisation_Shiny',
 'dev_language_MATLAB',
 'big_data_Oracle Database',
 'BI_tools_Google Data Studio',
 'big_data_MongoDB',
 'visualisation_Geoplotlib',
 'big_data_Google Cloud BigQuery',
 'IDE_MATLAB',
 'IDE_Other',
 'notebooks_AzureNotebooks',
 'notebooks_IBMWatsonStudio',
 'big_data_Microsoft Azure SQL Database',
 'visualisation_Bokeh',
 'dev_language_Bash',
 'sharing_Personalblog',
 'visualisation_Other',
 'big_data_Google Cloud SQL',
 'BI_tools_Qlik',
 'notebooks_Other',
 'BI_tools_Tableau CRM',
 'visualisation_D3 js',
 'visualisation_Leaflet / Folium',
 'big_data_Amazon Redshift',
 'IDE_None',
 'notebooks_DatabricksCollaborativeNotebooks',
 'BI_tools_Salesforce',
 'IDE_Vim / Emacs',
 'BI_tools_Other',
 'big_data_Snowflake',
 'notebooks_AmazonSagemakerStudioNotebooks',
 'big_data_Google Cloud BigTable',
 'big_data_IBM Db2',
 'sharing_Shiny',
 'big_data_Other',
 'big_data_Amazon RDS',
 'big_data_Microsoft Azure Cosmos DB',
 'dev_language_None',
 'sharing_Streamlit',
 'BI_tools_SAP Analytics Cloud',
 'sharing_Other',
 'big_data_Google Cloud Firestore',
 'BI_tools_Alteryx',
 'sharing_PlotlyDash',
 'visualisation_Altair',
 'BI_tools_Microsoft Azure Synapse',
 'notebooks_AmazonEMRNotebooks',
 'big_data_Amazon DynamoDB',
 'BI_tools_Looker',
 'big_data_Google Cloud Spanner',
 'BI_tools_Amazon QuickSight',
 'notebooks_ObservableNotebooks',
 'big_data_Amazon Aurora',
 'notebooks_Zeppelin/ZeplNotebooks',
 'sharing_NBViewer',
 'BI_tools_TIBCO Spotfire',
 'notebooks_DeepnoteNotebooks',
 'notebooks_Paperspace/Gradient',
 'notebooks_CodeOcean',
 'dev_language_Julia',
 'dev_language_Swift',
 'BI_tools_Domo',
 'BI_tools_Sisense',
 'BI_tools_Thoughtspot']

In [68]:
df_limitado = df_da.drop(columns=lista_drop, axis=1)

In [71]:
df_limitado_all_tools = columna_combo(df_limitado, ['dev_language_', 'IDE_', 'visualisation_', 'big_data_', 'BI_tools_', 'notebooks_'], 'combo_all_tools')

{'big_data_Microsoft SQL Server', 'IDE_Visual Studio', 'BI_tools_Microsoft Power BI', 'visualisation_Plotly / Plotly Express', 'big_data_MySQL', 'IDE_Jupyter (JupyterLab; Jupyter Notebooks; etc)', 'dev_language_R', 'combo_dev_lang_notebooks_', 'notebooks_ColabNotebooks', 'visualisation_Seaborn', 'IDE_Spyder', 'IDE_Visual Studio Code (VSCode)', 'IDE_Notepad++', 'IDE_RStudio', 'BI_tools_Tableau', 'dev_language_Java', 'visualisation_Ggplot / ggplot2', 'IDE_Jupyter Notebook', 'dev_language_Python', 'big_data_PostgreSQL', 'visualisation_None', 'visualisation_Matplotlib', 'dev_language_C', 'dev_language_SQL', 'notebooks_KaggleNotebooks', 'BI_tools_None', 'IDE_PyCharm', 'notebooks_None'}


,tools,count
0,,291
1,R;R;R;RStudio;Ggplot / ggplot2,12
2,Python;Seaborn;Jupyter Notebook;Python;Matplotlib,11
3,Python;SQL;Seaborn;Jupyter Notebook;Python;Matplotlib;SQL,11
4,SQL;SQL,11
5,Other,7
6,Colab Notebooks;Python;SQL;Colab Notebooks;Seaborn;Jupyter Notebook;Python;Matplotlib;SQL,7
7,Python;Python;PyCharm,7
8,R;R;R;SQL;RStudio;Ggplot / ggplot2;SQL,7
9,Python; Kaggle Notebooks;Seaborn;Jupyter Notebook;Python;Matplotlib; Kaggle Notebooks,7
